In [2]:
import lxml.html
import pandas as pd
import re
import requests

from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from pathlib import Path
from time import sleep

## Setup

In [68]:
ident = (
    "Stephanie Andrews (jellomoat@gmail.com), " + 
    "scraping for educational purposes"
)
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}

## Fetching and Cacheing functions

In [69]:
def get_next_url(count, fetch_limit=100, after_id=None):
    base_url = f"https://www.reddit.com/subreddits/popular/?limit={fetch_limit}&show=all"

    # handle if first page
    if not after_id:
        print(base_url)
        return base_url
    else:
        return f"{base_url}&count={count}&after=t{after_id}"

def fetch_next_page(page_nr, count, fetch_limit=100, after_id=None):
    url = get_next_url(count, fetch_limit, after_id)
    print(f"page: {page_nr}")
    print(f"Fetching {url}")
    try:
        return requests.get(
            url,
            headers=headers
        )
    except:
        print("No more pages!")

In [70]:
# page_nr = 0
# html = fetch_next_page(page_nr)

def fetch_pages(start_pg, end_pg, core_df):
    # setup
    pages_dir = "raw-pages-070723/"
    result_limit = 100
    results_count = 0

    # create or get fetch log
    log = Path(pages_dir + "log.csv")
    # create log df
    log_df = pd.read_csv(log)

    if log.exists():
        print("log exists")
    else:
        with open(log, "w") as l:
            print("test")
            l.write("fetch_dt, url, page_nr\n")

    # fetch each page
    # if last resultset returned less than the limit, stop fetching
    for page_nr in range(start_pg, end_pg): # swap to while loop after
        dest = Path(pages_dir + str(page_nr) + ".html")

        if dest.exists(): # load it from file
            print(f"Already have {dest}, loading!")
            file = open(dest, "r")
            page_html = file.read()
            file.close()

        else: # fetch it!
            last_page_id = core_df.loc[core_df["dt_retrieved"] == core_df["dt_retrieved"].max()]["page_id"].values[0] \
                if (len(core_df) > 1) else None
            print(f"page #{page_nr}: last page id => {last_page_id}")
            page_resp = fetch_next_page(page_nr, results_count, fetch_limit=100, after_id=last_page_id)
            page_html = page_resp.text
            sleep(2)

            # save to file
            with open(dest, "w") as f:
                f.write(page_html)

            with open(log, "a") as l:
                fetch_dt = datetime.now()
                url = page_resp.url
                l.write(",".join([str(fetch_dt), url, str(page_nr)]) + "\n")

        # parse each page, add to df
        soup = BeautifulSoup(page_html, "lxml")
        print(soup.title)

        r_els = soup.select("#siteTable > div.subreddit")
        core_df = parse_and_add_to_df(r_els, core_df, page_nr, log_df["fetch_dt"][page_nr])
        last_count = len(r_els)
        print(f"last_count: {last_count}")
        results_count += last_count
        print(f"results_count: {results_count}")
        print(f"df count: {len(core_df)}")
        if len(r_els) < 10:
            print(len(r_els))
            break
    return core_df

## Parsing functions

In [71]:
def parse_and_add_to_df(els_list, start_df, page_nr, fetch_dt):
    print(f"start count: {len(start_df)}")
    search_str = re.compile(r"a community for (.*)report")
    parsed_subs_list = []

    for r in els_list:
        sub_data = {
        "name": r.select_one(".titlerow").text.split(":")[0],
        "desc": get_all_desc(r.select("div.md > p")),
        "sub_age_tup": (re.search(search_str, r.text).group(1).split()),
        "page_id": r.get("id").lstrip("thing_"),
        "num_members": r.select_one("p.tagline > span.unvoted > span.number").text,
        "page_nr": page_nr,
        "date_retrieved": fetch_dt.split()[0]
        }
        sub_data["age_num"] = sub_data["sub_age_tup"][0]
        sub_data["age_word"] = sub_data["sub_age_tup"][1]
        sub_data.pop("sub_age_tup")
        parsed_subs_list.append(sub_data)

    end_df = pd.concat([start_df, pd.DataFrame(parsed_subs_list)], ignore_index=True)
    print(f"final core_df len: {len(end_df)}")
    return end_df

In [72]:
def get_all_desc(desc_elements_list):
    if len(desc_elements_list) > 0:
        return " ".join([el.text for el in desc_elements_list])
    else:
        ""

In [73]:
def convert_age_ago_to_created_ts(row):
    days_conversion = {
        "year": 365,
        "month": 30,
        "day": 1,
        "hour": (1/24)
    }
    days = days_conversion[row["age_word"].rstrip("s")] * int(row["age_num"])
    
    return datetime.strptime(row["date_retrieved"], "%Y-%m-%d") - timedelta(days=days)

In [74]:
def convert_large_nums(num):
    if num >= 1000000:
        return f"{num/1000000:.1f} million"
    elif num >= 1000:
        return f"{num/1000:.1f} thousand"
    else:
        return num

In [75]:
# Execute this in final run
# core_df["created_dt"] = core_df.apply(lambda x: convert_age_ago_to_created_ts(x).year, axis=1)

## Get and Parse Data

In [76]:
core_df = pd.DataFrame()
core_df = fetch_pages(0, 50, core_df)

log exists
Already have raw-pages-070723/0.html, loading!
<title>subreddits</title>
start count: 0
final core_df len: 100
last_count: 100
results_count: 100
df count: 100
Already have raw-pages-070723/1.html, loading!
<title>subreddits</title>
start count: 100
final core_df len: 200
last_count: 100
results_count: 200
df count: 200
Already have raw-pages-070723/2.html, loading!
<title>subreddits</title>
start count: 200
final core_df len: 300
last_count: 100
results_count: 300
df count: 300
Already have raw-pages-070723/3.html, loading!
<title>subreddits</title>
start count: 300
final core_df len: 400
last_count: 100
results_count: 400
df count: 400
Already have raw-pages-070723/4.html, loading!
<title>subreddits</title>
start count: 400
final core_df len: 500
last_count: 100
results_count: 500
df count: 500
Already have raw-pages-070723/5.html, loading!
<title>subreddits</title>
start count: 500
final core_df len: 600
last_count: 100
results_count: 600
df count: 600
Already have raw-pa

In [77]:
# core_df = fetch_pages(500, 1000, core_df)

In [78]:
# core_df = fetch_pages(1000, 1250, core_df)

In [79]:
# core_df = fetch_pages(1250, 1500, core_df)

In [80]:
# core_df = fetch_pages(1500, 1750, core_df)

In [81]:
# core_df.to_csv("./070623_df_0_1499_v2.csv")

In [82]:
len(core_df)
# html

4088

In [83]:
core_df.head()

,name,desc,page_id,num_members,page_nr,date_retrieved,age_num,age_word
0,r/Home,"Everything home related: interior design, home...",5_2qs0k,"135,237",0,2023-07-07,14,years
1,r/AskReddit,r/AskReddit is the place to ask and answer tho...,5_2qh1i,"41,833,971",0,2023-07-07,15,years
2,r/mildlyinfuriating,jugkfmghgug,5_2ubgg,"5,961,250",0,2023-07-07,11,years
3,r/facepalm,/r/facepalm has gone private in protest of the...,5_2r5rp,"7,469,361",0,2023-07-07,13,years
4,r/diablo4,Welcome to the un official Diablo 4 subreddit!...,5_2rzx9,"746,468",0,2023-07-07,12,years


In [84]:
core_df["created_dt"] = core_df.apply(lambda x: convert_age_ago_to_created_ts(x).year, axis=1)

In [85]:
core_df["human_num_members"] = core_df["num_members"].apply(lambda x: x.replace(",", "")).astype(int).apply(lambda x: convert_large_nums(x))

In [86]:
core_df.head()

,name,desc,page_id,num_members,page_nr,date_retrieved,age_num,age_word,created_dt,human_num_members
0,r/Home,"Everything home related: interior design, home...",5_2qs0k,"135,237",0,2023-07-07,14,years,2009,135.2 thousand
1,r/AskReddit,r/AskReddit is the place to ask and answer tho...,5_2qh1i,"41,833,971",0,2023-07-07,15,years,2008,41.8 million
2,r/mildlyinfuriating,jugkfmghgug,5_2ubgg,"5,961,250",0,2023-07-07,11,years,2012,6.0 million
3,r/facepalm,/r/facepalm has gone private in protest of the...,5_2r5rp,"7,469,361",0,2023-07-07,13,years,2010,7.5 million
4,r/diablo4,Welcome to the un official Diablo 4 subreddit!...,5_2rzx9,"746,468",0,2023-07-07,12,years,2011,746.5 thousand


In [87]:
core_df[0:100].to_csv("./subreddits_set_070723_dt_top_100_3.csv")

In [88]:
core_df[0:1000].to_csv("./subreddits_set_070723_dt_top_1000_3.csv")

In [89]:
core_df.to_csv("./subreddits_set_070723_dt_3.csv")

## Exploring the data...

In [185]:
# dom = lxml.html.fromstring(html)
html = fetch_next_page(0, 0).text
soup = BeautifulSoup(html, "lxml")
soup.title

https://www.reddit.com/subreddits/popular/?limit=100&show=all
page: 0
Fetching https://www.reddit.com/subreddits/popular/?limit=100&show=all


<title>subreddits</title>

In [186]:
# r_els = soup.cssselect("div > #siteTable")[0].cssselect("div > .subreddit")
# r_els = soup.find("div", id="siteTable").find_all("div", attrs={"class": "subreddit"})
# r_els = soup.select_one("div[id=siteTable]").find_all("div", attrs={"class": "subreddit"})
r_els = soup.select("#siteTable > div.subreddit")

len(r_els)

100

In [187]:
# lxml.html.tostring(r_els[0])
r_els[0]

<div class="thing id-t5_2qs0k odd subreddit" data-fullname="t5_2qs0k" data-gildings="0" data-type="subreddit" data-whitelist-status="all_ads" id="thing_t5_2qs0k" onclick="click_thing(this)"><p class="parent"></p><div class="midcol"><span class="fancy-toggle-button subscribe-button toggle" data-sr_name="Home" style=""><a class="option active add login-required" href="#" tabindex="100">join</a><a class="option remove" href="#">leave</a></span></div><div class="entry unvoted"><p class="titlerow"><a class="title" href="https://www.reddit.com/r/Home/">r/Home: Home</a></p><div class="description"><form action="#" class="usertext warn-on-unload" id="form-t5_2qs0kvqa" onsubmit="return post_form(this, 'editusertext')"><input name="thing_id" type="hidden" value="t5_2qs0k"/><div class="usertext-body may-blank-within md-container"><div class="md"><p>Everything home related: interior design, home improvement, architecture.</p>
</div>
</div></form></div><p class="tagline"><span class="score dislikes

In [197]:
# all text for a single subreddit element, incl children
# r_els[0].text_content()
r_els[0].text

'joinleaver/Home: HomeEverything home related: interior design, home improvement, architecture.\n\n135,308 subscribers135,309 subscribers135,310 subscribers, a community for 14 yearsreport'

In [198]:
# all text for all subreddits on page, incl children
# text_content = [r.text_content().lstrip("joinleave").rstrip("report") for r in r_els]
text_content = [r.text.lstrip("joinleave").rstrip("report") for r in r_els]
text_content[:2]

['r/Home: HomeEverything home related: interior design, home improvement, architecture.\n\n135,308 subscribers135,309 subscribers135,310 subscribers, a community for 14 years',
 'r/AskReddit: Ask Reddit...r/AskReddit is the place to ask and answer thought-provoking questions.\n\n41,836,673 subscribers41,836,674 subscribers41,836,675 subscribers, a community for 15 years']

In [199]:
test_parsing_df = pd.DataFrame()
test_parsing_df = parse_and_add_to_df(r_els, test_parsing_df, page_nr=0)

start count: 0
final core_df len: 100


In [203]:
test_parsing_df.tail(2)

,name,desc,page_id,num_subscribers,page_nr,dt_retrieved,age_num,age_word
98,r/FashionReps,Reddit's largest community for the discussion ...,5_31hcv,"1,325,116",0,2023-07-07 23:35:05.314047,9,years
99,r/maybemaybemaybe,For those videos that make you think maybe...,5_38e1l,"2,709,200",0,2023-07-07 23:35:05.314374,8,years


In [204]:
# core_df.loc[core_df["dt_retrieved"] == core_df["dt_retrieved"].max()]["page_id"]
last_page_id = test_parsing_df.loc[test_parsing_df["dt_retrieved"] == test_parsing_df["dt_retrieved"].max()]["page_id"].values[0] \
    if (len(test_parsing_df) > 1) else None
last_page_id

'5_38e1l'

In [205]:
# records with desc
test_parsing_df[~test_parsing_df["desc"].isna()].head(2)

,name,desc,page_id,num_subscribers,page_nr,dt_retrieved,age_num,age_word
0,r/Home,"Everything home related: interior design, home...",5_2qs0k,"135,309",0,2023-07-07 23:35:05.260853,14,years
1,r/AskReddit,r/AskReddit is the place to ask and answer tho...,5_2qh1i,"41,836,674",0,2023-07-07 23:35:05.261666,15,years


## Getting images!

In [90]:
core_df.head()

,name,desc,page_id,num_members,page_nr,date_retrieved,age_num,age_word,created_dt,human_num_members
0,r/Home,"Everything home related: interior design, home...",5_2qs0k,"135,237",0,2023-07-07,14,years,2009,135.2 thousand
1,r/AskReddit,r/AskReddit is the place to ask and answer tho...,5_2qh1i,"41,833,971",0,2023-07-07,15,years,2008,41.8 million
2,r/mildlyinfuriating,jugkfmghgug,5_2ubgg,"5,961,250",0,2023-07-07,11,years,2012,6.0 million
3,r/facepalm,/r/facepalm has gone private in protest of the...,5_2r5rp,"7,469,361",0,2023-07-07,13,years,2010,7.5 million
4,r/diablo4,Welcome to the un official Diablo 4 subreddit!...,5_2rzx9,"746,468",0,2023-07-07,12,years,2011,746.5 thousand


In [91]:
def linkify(row):
    return "".join((f"https://www.reddit.com/{row['name']}".strip()))

linkify(core_df.iloc[0])

'https://www.reddit.com/r/Home'

In [92]:
core_df["subreddit_url"] = core_df.apply(lambda x: linkify(x), axis=1)

In [93]:
core_df["subreddit_url"][2]

'https://www.reddit.com/r/mildlyinfuriating'

In [94]:
image_html = requests.get(core_df["subreddit_url"][2], headers=headers).text

In [95]:
soup = BeautifulSoup(image_html)

In [96]:
soup.title

<title>jukmifgguggh</title>

In [97]:
import re
img_re_str = "url\((.*)\)"
img_tag = soup.select("div.ListingLayout-outerContainer")[0].select("span")[0]["style"]
img_url = re.search(img_re_str, img_tag)[1].split("?")[0]
img_url

'https://styles.redditmedia.com/t5_2ubgg/styles/bannerBackgroundImage_qhj1ts036d811.jpg'

In [98]:
def get_image(row):
    image_html = requests.get(row["subreddit_url"].strip("\n"), headers=headers)
    print(image_html.url)
    try:
        soup = BeautifulSoup(image_html.text)
        print(soup.title)
        img_re_str = "url\((.*)\)"
        img_tag = soup.select("div.ListingLayout-outerContainer")[0].select("span")[0]["style"]
        img_url = re.search(img_re_str, img_tag)[1].split("?")[0]
        print(img_url)
#         time.sleep(1)
        return img_url
    except:
        return None

In [99]:
from tqdm.auto import tqdm
from tqdm.notebook import trange, tqdm
tqdm.pandas()

In [100]:
# testing on a small subset
test_df = core_df[:10].copy()
test_df.iloc[2]

name                                        r/mildlyinfuriating
desc                                                jugkfmghgug
page_id                                                 5_2ubgg
num_members                                           5,961,250
page_nr                                                       0
date_retrieved                                       2023-07-07
age_num                                                      11
age_word                                                  years
created_dt                                                 2012
human_num_members                                   6.0 million
subreddit_url        https://www.reddit.com/r/mildlyinfuriating
Name: 2, dtype: object

In [101]:
test_df["image_url"] = test_df.progress_apply(lambda x: get_image(x), axis=1)

  0%|          | 0/10 [00:00<?, ?it/s]

https://www.reddit.com/r/Home
<title>Home</title>
https://www.reddit.com/r/AskReddit
<title>Ask Reddit...</title>
https://www.reddit.com/r/mildlyinfuriating
<title>jukmifgguggh</title>
https://styles.redditmedia.com/t5_2ubgg/styles/bannerBackgroundImage_qhj1ts036d811.jpg
https://www.reddit.com/r/facepalm
<title>now double verified</title>
https://styles.redditmedia.com/t5_2r5rp/styles/bannerBackgroundImage_xjz99n5qt8p61.png
https://www.reddit.com/r/diablo4
<title>Diablo IV</title>
https://styles.redditmedia.com/t5_2rzx9/styles/bannerBackgroundImage_9omlpnmfus4a1.png
https://www.reddit.com/r/therewasanattempt
<title>There was an attempt..</title>
https://www.reddit.com/r/AmItheAsshole
<title>Am I the Asshole? </title>
https://styles.redditmedia.com/t5_2xhvq/styles/bannerBackgroundImage_nacao2gsxu321.jpg
https://www.reddit.com/r/Damnthatsinteresting
<title>Damn, that's interesting!</title>
https://www.reddit.com/r/worldnews
<title>World News</title>
https://styles.redditmedia.com/t5_2qh1

In [102]:
for img in test_df["image_url"]:
    print(img)

None
None
https://styles.redditmedia.com/t5_2ubgg/styles/bannerBackgroundImage_qhj1ts036d811.jpg
https://styles.redditmedia.com/t5_2r5rp/styles/bannerBackgroundImage_xjz99n5qt8p61.png
https://styles.redditmedia.com/t5_2rzx9/styles/bannerBackgroundImage_9omlpnmfus4a1.png
None
https://styles.redditmedia.com/t5_2xhvq/styles/bannerBackgroundImage_nacao2gsxu321.jpg
None
https://styles.redditmedia.com/t5_2qh13/styles/bannerBackgroundImage_5q0f5lsk6pu01.png
https://styles.redditmedia.com/t5_35n7t/styles/bannerBackgroundImage_1sm4lfrze6n71.png


In [103]:
# ok the big set now
core_df["image_url"] = core_df.progress_apply(lambda x: get_image(x), axis=1)

  0%|          | 0/4088 [00:00<?, ?it/s]

https://www.reddit.com/r/Home
<title>Home</title>
https://www.reddit.com/r/AskReddit
<title>Ask Reddit...</title>
https://www.reddit.com/r/mildlyinfuriating
<title>jukmifgguggh</title>
https://styles.redditmedia.com/t5_2ubgg/styles/bannerBackgroundImage_qhj1ts036d811.jpg
https://www.reddit.com/r/facepalm
<title>now double verified</title>
https://styles.redditmedia.com/t5_2r5rp/styles/bannerBackgroundImage_xjz99n5qt8p61.png
https://www.reddit.com/r/diablo4
<title>Diablo IV</title>
https://styles.redditmedia.com/t5_2rzx9/styles/bannerBackgroundImage_9omlpnmfus4a1.png
https://www.reddit.com/r/therewasanattempt
None
https://www.reddit.com/r/AmItheAsshole
<title>Am I the Asshole? </title>
https://styles.redditmedia.com/t5_2xhvq/styles/bannerBackgroundImage_nacao2gsxu321.jpg
https://www.reddit.com/r/Damnthatsinteresting
<title>Damn, that's interesting!</title>
https://www.reddit.com/r/worldnews
<title>World News</title>
https://styles.redditmedia.com/t5_2qh13/styles/bannerBackgroundImage_5q

https://www.reddit.com/r/Tinder
<title>Everything about Tinder</title>
https://www.reddit.com/r/OldSchoolCool/
<title>Reddit - Dive into anything</title>
https://www.reddit.com/r/doordash_drivers
<title>DoorDash Drivers</title>
https://styles.redditmedia.com/t5_3h9bt/styles/bannerBackgroundImage_os6fecsksl1a1.jpeg
https://www.reddit.com/r/MadeMeSmile
<title>MadeMeSmile</title>
https://www.reddit.com/r/Steam
<title>Steam on Reddit</title>
https://www.reddit.com/r/tumblr
<title>Tumblr</title>
https://www.reddit.com/r/pcgaming
<title>PC Gaming</title>
https://styles.redditmedia.com/t5_2qhfg/styles/bannerBackgroundImage_ossi3xflzj4b1.png
https://www.reddit.com/r/StupidFood
<title>StupidFood : Food. Point. Laugh.</title>
https://www.reddit.com/r/legaladvice
<title>Legal Advice ~ A place to get simple legal advice* </title>
https://www.reddit.com/r/manga
<title>/r/manga: manga, on reddit.</title>
https://www.reddit.com/r/HolUp
<title>Hol'Up, wait a minute!</title>
https://styles.redditmedia.

https://www.reddit.com/r/legaladvicecanada
<title>Legal Advice for Canadians.</title>
https://www.reddit.com/r/HonkaiStarRail_leaks/
<title>Reddit - Dive into anything</title>
https://www.reddit.com/r/Weird
<title>Weird</title>
https://www.reddit.com/r/skyrimmods
<title>TES V: Skyrim Modding Community</title>
https://styles.redditmedia.com/t5_2sqqh/styles/bannerBackgroundImage_imlg9ld5ugg01.jpg
https://www.reddit.com/r/cscareerquestions
<title>CS Career Questions protests reddit</title>
https://www.reddit.com/r/Destiny
<title>Official Destiny Subreddit</title>
https://styles.redditmedia.com/t5_2qnvz/styles/bannerBackgroundImage_sa17vf14qly91.jpg
https://www.reddit.com/r/shitposting
<title>r/shitposting</title>
https://styles.redditmedia.com/t5_2usxq/styles/bannerBackgroundImage_o9gp53ng3vra1.png
https://www.reddit.com/r/videos
<title>/r/videos: Democratically-Driven Until Democracy is Enhanced</title>
https://www.reddit.com/r/NintendoSwitch
<title>Nintendo Switch - News, Updates, and I

https://www.reddit.com/r/Showerthoughts
<title>Showerthoughts</title>
https://www.reddit.com/r/golf
<title>Golf </title>
https://styles.redditmedia.com/t5_2qhcs/styles/bannerBackgroundImage_k76hijfizeg01.jpg
https://www.reddit.com/r/aww
<title>A subreddit for cute and cuddly John Oliver pictures</title>
https://styles.redditmedia.com/t5_2qh1o/styles/bannerBackgroundImage_diek72j8077b1.jpg
https://www.reddit.com/r/EscapefromTarkov
<title>Escape From Tarkov</title>
https://styles.redditmedia.com/t5_3aq6o/styles/bannerBackgroundImage_t2344eap6q231.png
https://www.reddit.com/r/canada
<title>Canada</title>
https://www.reddit.com/r/dndmemes
<title>DnDMemes</title>
https://styles.redditmedia.com/t5_35g8o/styles/bannerBackgroundImage_v3lgn1cekoq61.png
https://www.reddit.com/r/formuladank
<title>formuladank</title>
https://styles.redditmedia.com/t5_3ndbi/styles/bannerBackgroundImage_ojl79s18tqn81.jpg
https://www.reddit.com/r/conspiracy/
<title>Reddit - Dive into anything</title>
https://www.red

...

https://www.reddit.com/r/AtlantaHawks
<title>The Upvote Factory</title>
https://styles.redditmedia.com/t5_2sgwl/styles/bannerBackgroundImage_ldnjga2rhe861.jpg
https://www.reddit.com/r/bigbangtheory
<title>Big Bang Theory</title>
https://styles.redditmedia.com/t5_2s85x/styles/bannerBackgroundImage_ftl5xemleet51.png
https://www.reddit.com/r/darknet
<title>Darknet</title>
https://styles.redditmedia.com/t5_2revo/styles/bannerBackgroundImage_nkuxkd9ux1r21.jpg
https://www.reddit.com/r/werkzaken/
<title>Reddit - Dive into anything</title>
https://www.reddit.com/r/theunforgiven/
<title>Reddit - Dive into anything</title>
https://www.reddit.com/r/LowSodiumDestiny
<title>All the Destiny, None of the Salt!</title>
https://styles.redditmedia.com/t5_3c0tx/styles/bannerBackgroundImage_qvty6uls5cy91.jpg
https://www.reddit.com/r/WeightLossAdvice
<title>WeightLossAdvice</title>
https://www.reddit.com/r/whatsapp/
<title>Reddit - Dive into anything</title>
https://www.reddit.com/r/BobsBurgers
<title>For 

https://www.reddit.com/r/ihadastroke
<title>ihaadastrokr</title>
https://styles.redditmedia.com/t5_hoj3z/styles/bannerBackgroundImage_fdl90hxj6es61.png
https://www.reddit.com/r/poppunkers
<title>/r/PopPunkers: discussion, news, songs, videos, reviews &amp; everything else Pop Punk!</title>
https://styles.redditmedia.com/t5_2sayb/styles/bannerBackgroundImage_0fo3cd33c5i01.png
https://www.reddit.com/r/BlackPeopleComedy
<title>BlackPeopleComedy</title>
https://styles.redditmedia.com/t5_673quk/styles/bannerBackgroundImage_iilcx2qnpcq91.jpeg
https://www.reddit.com/r/Infinity_For_Reddit
<title>Infinity_For_Reddit</title>
https://www.reddit.com/r/EnaiRim
<title>EnaiRim</title>
https://styles.redditmedia.com/t5_3n6pk/styles/bannerBackgroundImage_q3y0kvtnw9d41.png
https://www.reddit.com/r/budapest
<title>Budapest</title>
https://styles.redditmedia.com/t5_2rxz1/styles/bannerBackgroundImage_c2lh81wmn8d31.jpg
https://www.reddit.com/r/torontobiking
<title>Toronto Cycling</title>
https://www.reddit.

https://www.reddit.com/r/TheCivilService
<title>The UK Civil Service</title>
https://styles.redditmedia.com/t5_3k268/styles/bannerBackgroundImage_24i59d9odj941.jpg
https://www.reddit.com/r/armwrestling
<title>Arm Wrestling!</title>
https://styles.redditmedia.com/t5_2sw78/styles/bannerBackgroundImage_enlnzeqcfuka1.png
https://www.reddit.com/r/FixedGearBicycle
<title>Fixed Gear Bicycle</title>
https://styles.redditmedia.com/t5_2qo7a/styles/bannerBackgroundImage_8tzuoxm7vck01.png
https://www.reddit.com/r/anno1800
<title>Anno 1800</title>
https://www.reddit.com/r/GreenBayPackers
<title>Green Bay Packers</title>
https://styles.redditmedia.com/t5_2qqd2/styles/bannerBackgroundImage_gennu3gsyyoa1.jpg
https://www.reddit.com/r/Palestine
<title>Palestine</title>
https://styles.redditmedia.com/t5_2qhak/styles/bannerBackgroundImage_z73z6muhkh011.png
https://www.reddit.com/r/jakertown
<title>jakertown</title>
https://www.reddit.com/r/LadiesOfWrestling
<title>Ladies of Wrestling</title>
https://style

https://www.reddit.com/r/rawdenim/
<title>Reddit - Dive into anything</title>
https://www.reddit.com/r/mensfashion
<title>Men's Fashion</title>
https://www.reddit.com/r/WindowsOnDeck
<title>WindowsOnDeck</title>
https://www.reddit.com/r/D4Sorceress
<title>Diablo IV Sorceress</title>
https://styles.redditmedia.com/t5_2tma7u/styles/bannerBackgroundImage_8i22j0y06m851.png
https://www.reddit.com/r/TheGriffonsSaddlebag
<title>TheGriffonsSaddlebag</title>
https://styles.redditmedia.com/t5_w4xiq/styles/bannerBackgroundImage_g2bhkfsyy8x21.png
https://www.reddit.com/r/physicsmemes
<title>Physics Memes</title>
https://www.reddit.com/r/WarhammerFantasy
<title>Warhammer Fantasy</title>
https://styles.redditmedia.com/t5_2rv4g/styles/bannerBackgroundImage_225f0blg2sl51.jpg
https://www.reddit.com/r/StremioAddons
<title>Stremio Addons</title>
https://www.reddit.com/r/InfowarriorRides
<title>The Cars of Infowars</title>
https://www.reddit.com/r/Soundbars
<title>Discussion about soundbars</title>
https:

https://www.reddit.com/r/literature
<title>literature</title>
https://www.reddit.com/r/fragranceclones
<title>Fragrance Clones / Inspired by </title>
https://www.reddit.com/r/tankiejerk
<title>TankieJerk: Revengeance</title>
https://styles.redditmedia.com/t5_136u9r/styles/bannerBackgroundImage_p48i0ts82mt71.png
https://www.reddit.com/r/DerechoGenial
<title>ELI5s de derecho, actualidad, consultas y todo lo que tenga que ver con el derecho y la abogacía.</title>
https://styles.redditmedia.com/t5_q0war/styles/bannerBackgroundImage_vhzd56le8yh91.png
https://www.reddit.com/r/sims2
<title>The Sims 2</title>
https://www.reddit.com/r/Houdini
<title>Houdini</title>
https://www.reddit.com/r/traditionaltattoos
<title>Traditional American Tattoos</title>
https://www.reddit.com/r/tbatenovel
<title>The Beginning After The End</title>
https://styles.redditmedia.com/t5_yxu31/styles/bannerBackgroundImage_gzswihg7jgu71.png
https://www.reddit.com/r/Stormgate
<title>Stormgate</title>
https://styles.reddit

In [104]:
core_df[:100]["image_url"].value_counts()

image_url
https://styles.redditmedia.com/t5_2ubgg/styles/bannerBackgroundImage_qhj1ts036d811.jpg     1
https://styles.redditmedia.com/t5_2r5rp/styles/bannerBackgroundImage_xjz99n5qt8p61.png     1
https://styles.redditmedia.com/t5_2s5oq/styles/bannerBackgroundImage_ozlnzosznvg91.png     1
https://styles.redditmedia.com/t5_2tk0s/styles/bannerBackgroundImage_98c72voec68b1.png     1
https://styles.redditmedia.com/t5_2rnve/styles/bannerBackgroundImage_2q1u6ier9cpa1.png     1
https://styles.redditmedia.com/t5_2to41/styles/bannerBackgroundImage_be0j6czrjnf21.png     1
https://styles.redditmedia.com/t5_2r0cn/styles/bannerBackgroundImage_1npb2gkjly981.png     1
https://styles.redditmedia.com/t5_2r2jt/styles/bannerBackgroundImage_r1fezkylmy251.png     1
https://styles.redditmedia.com/t5_4q5zoc/styles/bannerBackgroundImage_ocjmysayb36b1.png    1
https://styles.redditmedia.com/t5_2rgs7/styles/bannerBackgroundImage_pc63neps4dra1.jpg     1
https://styles.redditmedia.com/t5_2wbww/styles/bannerBackgro

In [105]:
len(core_df[~core_df["image_url"].isna()])

2513

In [106]:
# doing this to render in flourish tooltip only when image_url is available
img_html_for_flourish = "".join(['<img src="{{image_url}}" ', 
                                 'style="', 'height: 50px; width:100%; object-fit: cover;">',])
img_html_for_flourish

'<img src="{{image_url}}" style="height: 50px; width:100%; object-fit: cover;">'

In [107]:
def get_flourish_img_html(row):
    if row["image_url"]:
        return img_html_for_flourish.replace("{{image_url}}", row["image_url"])
    else:
        return None

core_df["flourish_img_html"] = core_df.apply(lambda x: get_flourish_img_html(x), axis=1)

In [108]:
for x in core_df[:10]["flourish_img_html"]:
    print(x)

None
None
<img src="https://styles.redditmedia.com/t5_2ubgg/styles/bannerBackgroundImage_qhj1ts036d811.jpg" style="height: 50px; width:100%; object-fit: cover;">
<img src="https://styles.redditmedia.com/t5_2r5rp/styles/bannerBackgroundImage_xjz99n5qt8p61.png" style="height: 50px; width:100%; object-fit: cover;">
<img src="https://styles.redditmedia.com/t5_2rzx9/styles/bannerBackgroundImage_9omlpnmfus4a1.png" style="height: 50px; width:100%; object-fit: cover;">
None
<img src="https://styles.redditmedia.com/t5_2xhvq/styles/bannerBackgroundImage_nacao2gsxu321.jpg" style="height: 50px; width:100%; object-fit: cover;">
None
<img src="https://styles.redditmedia.com/t5_2qh13/styles/bannerBackgroundImage_5q0f5lsk6pu01.png" style="height: 50px; width:100%; object-fit: cover;">
<img src="https://styles.redditmedia.com/t5_35n7t/styles/bannerBackgroundImage_1sm4lfrze6n71.png" style="height: 50px; width:100%; object-fit: cover;">


In [109]:
core_df.head(5)

,name,desc,page_id,num_members,page_nr,date_retrieved,age_num,age_word,created_dt,human_num_members,subreddit_url,image_url,flourish_img_html
0,r/Home,"Everything home related: interior design, home...",5_2qs0k,"135,237",0,2023-07-07,14,years,2009,135.2 thousand,https://www.reddit.com/r/Home,None,None
1,r/AskReddit,r/AskReddit is the place to ask and answer tho...,5_2qh1i,"41,833,971",0,2023-07-07,15,years,2008,41.8 million,https://www.reddit.com/r/AskReddit,None,None
2,r/mildlyinfuriating,jugkfmghgug,5_2ubgg,"5,961,250",0,2023-07-07,11,years,2012,6.0 million,https://www.reddit.com/r/mildlyinfuriating,https://styles.redditmedia.com/t5_2ubgg/styles...,"<img src=""https://styles.redditmedia.com/t5_2u..."
3,r/facepalm,/r/facepalm has gone private in protest of the...,5_2r5rp,"7,469,361",0,2023-07-07,13,years,2010,7.5 million,https://www.reddit.com/r/facepalm,https://styles.redditmedia.com/t5_2r5rp/styles...,"<img src=""https://styles.redditmedia.com/t5_2r..."
4,r/diablo4,Welcome to the un official Diablo 4 subreddit!...,5_2rzx9,"746,468",0,2023-07-07,12,years,2011,746.5 thousand,https://www.reddit.com/r/diablo4,https://styles.redditmedia.com/t5_2rzx9/styles...,"<img src=""https://styles.redditmedia.com/t5_2r..."


In [110]:
core_df["rank_str"] = core_df.index.astype("str")

In [111]:
core_df[:100].to_csv("./df_070723_100_with_images_3.csv")

In [112]:
core_df[:1000].to_csv("./df_070723_1k_with_images_3.csv")

In [113]:
core_df.to_csv("./df_070723_with_images_3.csv")

In [114]:
core_df

,name,desc,page_id,num_members,page_nr,date_retrieved,age_num,age_word,created_dt,human_num_members,subreddit_url,image_url,flourish_img_html,rank_str
0,r/Home,"Everything home related: interior design, home...",5_2qs0k,"135,237",0,2023-07-07,14,years,2009,135.2 thousand,https://www.reddit.com/r/Home,None,None,0
1,r/AskReddit,r/AskReddit is the place to ask and answer tho...,5_2qh1i,"41,833,971",0,2023-07-07,15,years,2008,41.8 million,https://www.reddit.com/r/AskReddit,None,None,1
2,r/mildlyinfuriating,jugkfmghgug,5_2ubgg,"5,961,250",0,2023-07-07,11,years,2012,6.0 million,https://www.reddit.com/r/mildlyinfuriating,https://styles.redditmedia.com/t5_2ubgg/styles...,"<img src=""https://styles.redditmedia.com/t5_2u...",2
3,r/facepalm,/r/facepalm has gone private in protest of the...,5_2r5rp,"7,469,361",0,2023-07-07,13,years,2010,7.5 million,https://www.reddit.com/r/facepalm,https://styles.redditmedia.com/t5_2r5rp/styles...,"<img src=""https://styles.redditmedia.com/t5_2r...",3
4,r/diablo4,Welcome to the un official Diablo 4 subreddit!...,5_2rzx9,"746,468",0,2023-07-07,12,years,2011,746.5 thousand,https://www.reddit.com/r/diablo4,https://styles.redditmedia.com/t5_2rzx9/styles...,"<img src=""https://styles.redditmedia.com/t5_2r...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4083,r/vintageaudio,r/VintageAudio NEW! Join the rest of the Redd...,5_2svvy,"87,062",40,2023-07-07,11,years,2012,87.1 thousand,https://www.reddit.com/r/vintageaudio,None,None,4083
4084,r/optometry,Welcome to /r/optometry! This subreddit for ey...,5_2shdt,"34,103",40,2023-07-07,12,years,2011,34.1 thousand,https://www.reddit.com/r/optometry,None,None,4084
4085,r/EmuDeck,Play all your retro games on the Steam Deck We...,5_698bu8,"13,260",40,2023-07-07,1,year,2022,13.3 thousand,https://www.reddit.com/r/EmuDeck,None,None,4085
4086,r/KimKardashianPics,"Dedicated to pictures of Kim Kardashian, regul...",5_2uj0t,"236,356",40,2023-07-07,10,years,2013,236.4 thousand,https://www.reddit.com/r/KimKardashianPics,None,None,4086


## Datawrapper File Prep

In [8]:
dw_df = pd.read_csv("./output/top_1000_with_categories_3.csv")

In [45]:
dw_df.iloc[:, [1, 2, 4, 9, 31]].to_csv("./output/datawrapper_output_070723_dt_top_1000_3.csv")

In [47]:
dw_df_loaded = pd.read_csv("./output/datawrapper_output_070723_dt_top_1000_3.csv")

In [48]:
len(dw_df_loaded)

1000